## 🎯 Objetivo del Proyecto

El objetivo de este análisis es extraer información útil desde una base de datos relacional de una plataforma de lectura digital, con el fin de apoyar el desarrollo de una propuesta de valor para un nuevo producto. 

Se estudiarán aspectos como:
- Volumen de libros publicados recientemente
- Participación de editoriales
- Reseñas y calificaciones de los usuarios
- Autores con mejor desempeño

## 🔌 Conexión a la Base de Datos

Usaremos SQLAlchemy para conectarnos a la base de datos PostgreSQL proporcionada por TripleTen. Recuerda que el archivo `CA.pem` debe estar en la misma carpeta del proyecto para habilitar la conexión segura.


In [18]:
# importar librerías
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})


## 🗃️ Exploración de Tablas
Se consultan las primeras filas de cada tabla para conocer su estructura y los tipos de datos.

In [6]:
# Ver primeras filas de cada tabla
print(pd.read_sql('SELECT * FROM books LIMIT 5', con=engine))
print(pd.read_sql('SELECT * FROM authors LIMIT 5', con=engine))
print(pd.read_sql('SELECT * FROM publishers LIMIT 5', con=engine))
print(pd.read_sql('SELECT * FROM ratings LIMIT 5', con=engine))
print(pd.read_sql('SELECT * FROM reviews LIMIT 5', con=engine))


   book_id  author_id                                              title  \
0        1        546                                       'Salem's Lot   
1        2        465                 1 000 Places to See Before You Die   
2        3        407  13 Little Blue Envelopes (Little Blue Envelope...   
3        4         82  1491: New Revelations of the Americas Before C...   
4        5        125                                               1776   

   num_pages publication_date  publisher_id  
0        594       2005-11-01            93  
1        992       2003-05-22           336  
2        322       2010-12-21           135  
3        541       2006-10-10           309  
4        386       2006-07-04           268  
   author_id                          author
0          1                      A.S. Byatt
1          2  Aesop/Laura Harris/Laura Gibbs
2          3                 Agatha Christie
3          4                   Alan Brennert
4          5        Alan Moore/David   Llo

## 🔍 Consultas y Resultados
A continuación, se realizan las consultas solicitadas en el proyecto.

### 1. Libros publicados después del 1 de enero de 2000

**Consulta:** Se seleccionan libros cuya fecha de publicación sea posterior a '2000-01-01'.
**Objetivo:** Conocer el volumen de libros recientes en la plataforma.

In [13]:
query1 = """
SELECT COUNT(*) AS books_after_2000
FROM books
WHERE publication_date > '2000-01-01'
"""
pd.read_sql(query1, con=engine)


,books_after_2000
0,819


**Conclusión:** Esta métrica ayuda a identificar cuántos títulos recientes se tienen en el catálogo, ideal para enfocar una app en contenido moderno.


### 2. Número de reseñas de usuarios y calificación promedio por libro

**Consulta:** Se agrupan los libros y se cuentan las reseñas, además de calcular la calificación promedio.
**Objetivo:** Identificar qué libros generan mayor interacción y satisfacción.

In [14]:
query2 = """
SELECT 
    b.title,
    COUNT(DISTINCT r.review_id) AS num_reviews,
    ROUND(AVG(rt.rating), 2) AS avg_rating
FROM books b
LEFT JOIN reviews r ON b.book_id = r.book_id
LEFT JOIN ratings rt ON b.book_id = rt.book_id
GROUP BY b.title
ORDER BY num_reviews DESC
"""
pd.read_sql(query2, con=engine)


,title,num_reviews,avg_rating
0,Memoirs of a Geisha,8,4.14
1,Twilight (Twilight #1),7,3.66
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
3,The Glass Castle,6,4.21
4,Eat Pray Love,6,3.40
...,...,...,...
994,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67
995,The Natural Way to Draw,0,3.00
996,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
997,Essential Tales and Poems,0,4.00


**Conclusión:** Este análisis revela los libros más populares y mejor valorados, lo que puede usarse para destacar títulos en la app.


### 3. Editorial con más libros de más de 50 páginas

**Consulta:** Se filtran libros con más de 50 páginas, se agrupan por editorial y se cuentan.
**Objetivo:** Encontrar editoriales que publiquen libros relevantes y no folletos.

In [15]:
query3 = """
SELECT 
    p.publisher,
    COUNT(b.book_id) AS num_books
FROM books b
JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY num_books DESC
LIMIT 1
"""
pd.read_sql(query3, con=engine)

,publisher,num_books
0,Penguin Books,42


**Conclusión:** Identifica a la editorial con mayor volumen de libros significativos, útil para alianzas estratégicas o recomendaciones.


### 4. Autor con mejor calificación promedio (con mínimo 50 ratings)

**Consulta:** Se filtran libros que tienen al menos 50 calificaciones y se calcula el promedio por autor.
**Objetivo:** Destacar autores con mejor desempeño según la comunidad.

In [16]:
query4 = """
SELECT 
    a.author,
    ROUND(AVG(rt.rating), 2) AS avg_rating
FROM books b
JOIN authors a ON b.author_id = a.author_id
JOIN ratings rt ON b.book_id = rt.book_id
GROUP BY a.author
HAVING COUNT(rt.rating) >= 50
ORDER BY avg_rating DESC
LIMIT 1
"""
pd.read_sql(query4, con=engine)


,author,avg_rating
0,Diana Gabaldon,4.3


**Conclusión:** Muestra el autor más valorado con suficiente base estadística. Ideal para destacar autores clave en el marketing del producto.


### 5. Promedio de reseñas de texto de usuarios que calificaron más de 50 libros

**Consulta:** Se identifican usuarios que han calificado más de 50 libros y se calcula el promedio de reseñas de texto.
**Objetivo:** Analizar el comportamiento de los usuarios más activos.

In [17]:
query5 = """
WITH active_users AS (
    SELECT username
    FROM ratings
    GROUP BY username
    HAVING COUNT(rating_id) > 50
)

SELECT 
    ROUND(AVG(review_count), 2) AS avg_text_reviews
FROM (
    SELECT username, COUNT(review_id) AS review_count
    FROM reviews
    WHERE username IN (SELECT username FROM active_users)
    GROUP BY username
) AS user_reviews
"""
pd.read_sql(query5, con=engine)



,avg_text_reviews
0,24.33


**Conclusión:** Permite conocer el nivel de compromiso de usuarios activos, clave para diseñar funciones de comunidad o incentivos para reseñas.


## 🧾 Conclusiones Generales
- La mayoría de los libros recientes (después del 2000) están bien representados en la base de datos.
- Algunos libros reciben muchas reseñas y calificaciones, lo que ayuda a entender las preferencias de los usuarios.
- Hay una editorial destacada en cuanto a volumen de libros sustanciales.
- Se identifican autores con alto desempeño basados en calificaciones.
- Los usuarios más activos también tienden a escribir más reseñas de texto.

Estos hallazgos pueden guiar el desarrollo de funcionalidades dentro de una app de lectura o comunidad literaria.
